In [ ]:
import io
import cv2
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image

def showarray(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    


In [ ]:
import cv2
import numpy as np
import math
import statistics


img=cv2.imread('Picture1.png') #path to the image
img = cv2.resize(img,(int(img.shape[1]),int(img.shape[0])))

def edges(img, k, s):
    kernel_sharpening = -s * np.ones([k,k])
    kernel_sharpening[k//2,k//2] = s * (k**2-1)+1
    img = cv2.filter2D(img, -1, kernel_sharpening)
    return(img)

In [ ]:
class CropLayer(object):
    def __init__(self, params, blobs):
        self.xstart = 0
        self.xend = 0
        self.ystart = 0
        self.yend = 0

    # Our layer receives two inputs. We need to crop the first input blob
    # to match a shape of the second one (keeping batch size and number of channels)
    def getMemoryShapes(self, inputs):
        inputShape, targetShape = inputs[0], inputs[1]
        batchSize, numChannels = inputShape[0], inputShape[1]
        height, width = targetShape[2], targetShape[3]

        self.ystart = (inputShape[2] - targetShape[2]) // 2
        self.xstart = (inputShape[3] - targetShape[3]) // 2
        self.yend = self.ystart + height
        self.xend = self.xstart + width

        return [[batchSize, numChannels, height, width]]

    def forward(self, inputs):
        return [inputs[0][:,:,self.ystart:self.yend,self.xstart:self.xend]]

In [ ]:
cv2.dnn_registerLayer('Crop', CropLayer)

In [ ]:
net = cv2.dnn.readNet(cv2.samples.findFile("deploy.prototxt"), cv2.samples.findFile("hed_pretrained_bsds.caffemodel"))

In [ ]:
inp = cv2.dnn.blobFromImage(cv2.UMat(img), scalefactor=1, size=(img.shape[1],img.shape[0]), 
                            mean=(104.00698793, 116.66876762, 122.67891434),
                            swapRB=False, crop=False)
net.setInput(inp)

out = net.forward()
out = out[0, 0]
mask = np.uint8((out>0.001)*255)
#img1 = cv2.merge((img[:,:,0]&mask,img[:,:,1]&mask,img[:,:,2]&mask))
showarray(mask)

In [ ]:
kernel_sharpening = -100 * np.ones([3,3])
kernel_sharpening[3//2,3//2] = 100 * (3**2-1)+1
print(kernel_sharpening)

In [ ]:
#kh = np.array([[-100,-100,-100],[-100,100*8+1,-100],[-100,-100,-100]])
#himg = cv2.filter2D(img,-1,kh)

clahe = cv2.createCLAHE(clipLimit=0, tileGridSize=(3,3))
img1 = img
#img1 = cv2.resize(img1,(int(img1.shape[1]*4),int(img1.shape[0]*4)))

#img1 = ~cv2.stylization(img1, sigma_s=1, sigma_r=0.0001)
#img1 = cv2.edgePreservingFilter(img1, flags=2, sigma_s=200, sigma_r=1)
img1 = np.uint8(((img1/255)**(0.5))*255)
#img1 = ~cv2.stylization(img1, sigma_s=1, sigma_r=0.0001)
#img1 = ~edges(img1,7,-1)
#img1 = cv2.edgePreservingFilter(img1, flags=2, sigma_s=200, sigma_r=1)
#img1 = ~edges(img1,7,-1)
#img1 = cv2.edgePreservingFilter(img1, flags=2, sigma_s=200, sigma_r=1)
#img1 = cv2.edgePreservingFilter(img1, flags=1, sigma_s=200, sigma_r=1)
#img1 = cv2.merge((clahe.apply(img1[:,:,0]),clahe.apply(img1[:,:,1]),clahe.apply(img1[:,:,2])))
#img1 = cv2.resize(img1,(int(img1.shape[1]//4),int(img1.shape[0]//4)))

#clahe = cv2.createCLAHE(clipLimit=0, tileGridSize=(7,7))
#img1 = cv2.merge((clahe.apply(img1[:,:,0]),clahe.apply(img1[:,:,1]),clahe.apply(img1[:,:,2])))


#img1 = np.uint8((img1>0)*255)#edges(img,1,-1)
#img1 = edges(img1,7,1)
#img1 = cv2.bilateralFilter(img1,3,75,75)



#img1 = ~edges(img1,3,1)
#img1 = ~edges(img1,7,1)
#img1 = ~edges(img1,7,1)
#img1 = np.uint8(np.ceil(img1/255)*255)
#img1 = img1[:,:,0]|img1[:,:,1]|img1[:,:,2]
#img1 = cv2.Canny(img1,0,0)
showarray(img1)


In [ ]:
img=cv2.imread('Picture1.png') #path to the image
img = cv2.resize(img,(int(img.shape[1]),int(img.shape[0])))


saliency = cv2.saliency.StaticSaliencyFineGrained_create()
(success, saliencyMap) = saliency.computeSaliency(edges(img,7,1))
mask = ~np.uint8(saliencyMap*255)


img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
l,a,b = cv2.split(img)
clahe = cv2.createCLAHE(clipLimit=0, tileGridSize=(3,3))
l = clahe.apply(mask)
img = cv2.merge((l,a,b))
img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)

showarray(img)

In [ ]:
import glob
curr_path = sorted(glob.glob("{}/*".format("20190411_day_1_only_smartphone")))
#curr_path = sorted(glob.glob("D:\\ML\\proj\\WM_Gasprom\\danilovsky\\*"))
print(curr_path)

In [ ]:
cap = cv2.VideoCapture(curr_path[1])
print(curr_path[1])

def warp(img1, img2, upscale = 4, downscale = 4):
    img1_shape = img1.shape
    img2_shape = img2.shape
    
    img1 = cv2.UMat(img1)
    img2 = cv2.UMat(img2)
    
    img1 = cv2.resize(img1,(int(img1_shape[1]*upscale),int(img1_shape[0]*upscale)))
    img2 = cv2.resize(img2,(int(img2_shape[1]*upscale),int(img2_shape[0]*upscale)))
    
    kaze = cv2.AKAZE_create(descriptor_type = cv2.AKAZE_DESCRIPTOR_MLDB)
    kaze.getDescriptorType()

    kp1, des1 = kaze.detectAndCompute(img1, None)
    kp2, des2 = kaze.detectAndCompute(img2, None)
    des1 = np.float32(des1.get())
    des2 = np.float32(des2.get())
    print(des1.shape[0], des2.shape[0])

    index_params = dict(algorithm = 1, trees = 1)
    search_params = dict(checks = 3)
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    if(des1.shape[0] > des2.shape[0]):
        des1,des2 = des2,des1
        kp1,kp2 = kp2,kp1
        img1,img2 = img2,img1


    matches = matcher.knnMatch(des1, des2, 2)
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.99*n.distance:
            good.append(m)

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good
                                      ]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good
                                      ]).reshape(-1, 1, 2)
    
    img1 = cv2.drawKeypoints(img1.get(), [kp1[m.queryIdx] for m in good], None, color=(0,255,0), flags=0)
    img2 = cv2.drawKeypoints(img2.get(), [kp2[m.trainIdx] for m in good], None, color=(0,255,0), flags=0)

    print(len(good))

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # see https://ch.mathworks.com/help/images/examples/find-image-rotation-and-scale-using-automated-feature-matching.html for details
    ss = M[0, 1]
    sc = M[0, 0]
    scaleRecovered = math.sqrt(ss * ss + sc * sc)
    thetaRecovered = math.atan2(ss, sc) * 180 / math.pi
    #print("MAP: Calculated scale difference: %.2f, "
    #                "Calculated rotation difference: %.2f" %
    #                (scaleRecovered, thetaRecovered))
    
    #deskew image
    im_out = cv2.warpPerspective(img2, np.linalg.inv(M),
        (int(img1_shape[1]*upscale), int(img1_shape[0]*upscale)))
    
    img1 = cv2.resize(img1,(int(img1_shape[1]/downscale),int(img1_shape[0]/downscale)))
    img2 = cv2.resize(img2,(int(img2_shape[1]/downscale),int(img2_shape[0]/downscale)))
    im_out = cv2.resize(im_out,(int(img1_shape[1]/downscale),int(img1_shape[0]/downscale)))

    return img1, img2, im_out
    

In [ ]:
from scipy.signal import convolve2d
def canney(img, downscale, final_downscale,low = 0, high = 0):
    img_shape = img.shape
    img = cv2.resize(img,(int(img_shape[1]//downscale),int(img_shape[0]//downscale)))
    img1 = cv2.Canny(img,low,high)
    X = img1//255

    X = X*convolve2d(X,np.array([[1,1,1], [1,0,1],[1,1,1]]),mode='same')
    X = np.uint8((X>0)*(255))
    X = cv2.resize(X,(int(img_shape[1]//final_downscale),int(img_shape[0]//final_downscale)))
    X = np.uint8((X>1)*(255))
    return(X)

def pir_canney(img, start_downscale, stop_downscale, final_downscale):
    img_shape = img.shape
    canney_final = canney(img,start_downscale,stop_downscale)
    for dw in range(start_downscale+1,stop_downscale):
        canney_final = canney_final//2+canney(img,dw,stop_downscale)//2
    
    canney_final = cv2.resize(canney_final,(int(img_shape[1]//final_downscale),int(img_shape[0]//final_downscale)))
    return(canney_final)

In [ ]:
from IPython.display import clear_output

import imutils

import matplotlib.pyplot as plt
from skimage import measure

downscale = 2

try:
    
    ret, img = cap.read()
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    subtractor0 = cv2.bgsegm.createBackgroundSubtractorCNT(minPixelStability = 20, maxPixelStability = 100, isParallel=True)
    #subtractor1 = cv2.bgsegm.createBackgroundSubtractorCNT(minPixelStability = 20, maxPixelStability = 101, isParallel=True)
    #subtractor2 = cv2.bgsegm.createBackgroundSubtractorCNT(minPixelStability = 20, maxPixelStability = 101, isParallel=True)
    #saliency = cv2.saliency.MotionSaliencyBinWangApr2014_create()
    img = cv2.resize(img,(int(img.shape[1]//2),int(img.shape[0]//2)))
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    #saliency.setImagesize(img.shape[1], img.shape[0])
    #saliency.init()
    while(True):
        #img_old = img.copy()
        
        ret, img = cap.read()
        
        
        
        img = cv2.resize(img,(int(img.shape[1]//2),int(img.shape[0]//2)))
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        
        (success, saliencyMap) = saliency.computeSaliency(edges(img,3,-1))
        
        mask = ~np.uint8(saliencyMap*255)


        img1 = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l,a,b = cv2.split(img1)
        l = mask
        
        clahe = cv2.createCLAHE(clipLimit=0, tileGridSize=(3,3))
        l = clahe.apply(l)
        l = subtractor0.apply(l)
        img1 = cv2.merge((l,a,b))
        img1 = cv2.cvtColor(img1, cv2.COLOR_LAB2BGR)
        
        
        
        cv2.imshow("new", np.vstack((img,img1,img1)))
        
        if cv2.waitKey(1) & 0xFF == 27:
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
except Exception as e:
    print(e)
    cv2.destroyAllWindows()
    cv2.waitKey(1)